In [3]:
# This notebook is  used to evaluate perofrmance of our RAG ingestion and query pipeline.
# Method:
#   - Generated test data using a sample GitHub repository (offline process not in this notebook)
#     - https://docs.ragas.io/en/stable/getstarted/rag_testset_generation/
#   - Use our ingestion pipeline to parse the same reporisiroty and index in our vector store
#   - Use RAGAS on teadt dataset + our answers fromn our RAG and visualise the mterics.
import os
import sys
import pandas as pd

sys.path.insert(1, '/home/jovyan/work/code')
from config import VectorDBConfig, ModelConfig
from config_helper import ConfigHelper
from ingestion_pipeline import IngestionPipeline
from localrag import LocalRAG

from trace_setup import get_tracer, get_logger
logger = get_logger()
tracer = get_tracer()

config_helper = ConfigHelper(False)

In [2]:
# Ingest a GitHub repository and import into our vector store
# print(config_helper.embedding_config)
print(config_helper.vector_db_config)
# Initialize the pipeline
pipeline = IngestionPipeline(
    vector_db_config=config_helper.vector_db_config,
    embedding_config=config_helper.embedding_config)

local_ingestion_file="data/dotnet-docs-aspire.txt"
repository="https://github.com/dotnet/docs-aspire"
with tracer.start_as_current_span(f"Starting ingesting {repository} using local ingestion file {local_ingestion_file}"):
    pipeline.process_single_file(local_ingestion_file,repository, True)


VectorDBConfig(url='http://qdrant:6333', api_key='zCxfEnC2RPCH1r64RPqtnY', collection_name='mistral-nomic-embed-text', vector_name='page_content_vector')


/home/jovyan/work/code/ingestion_pipeline.py:31: UserWarning: Api key is used with an insecure connection.
  self.qdrant = QdrantClient(url=vector_db_config.url, api_key=vector_db_config.api_key)


Total parts found: 363
Total files processed: 181
Sample file paths:
 1. README.md
 2. CODE_OF_CONDUCT.md
 3. SECURITY.md
Added 50 chunks for file docs/reference/aspire-faq.yml


ResponseHandlingException: Server disconnected without sending a response.

In [ ]:
# Test our RAG Solution

def demonstrate_local_rag(rag):
    """Demonstrate how to use the LocalRAG class."""    
    # Example questions to test
    questions = [
        "If I use Docker Compose, why should I care about .Net Aspire?",
        "Is .Net Aspire an alternative to Kubernetes?"
    ]
    with tracer.start_as_current_span("Entering questions loop."):
        for question in questions:
            print(f"Question: {question}")
            # print("\nRelevant chunks:")
            # with tracer.start_as_current_span("rag get chunks"):
            #     chunks = rag.get_relevant_chunks(question, k=5)
            #     for i, chunk in enumerate(chunks, 1):
            #         print(chunk)
            #         break
#                    print(f"\nChunk {i}:")
#                    print(f"Source: {chunk.metadata.get('file_path', 'Unknown')}")
#                    print(f"Content: {chunk.page_content[:200]}...")
                    
            print("\nGenerated Answer:")
            with tracer.start_as_current_span("Retrieve answers."):
                answer = rag.retrieve_and_answer(question, k=10)
                print(answer)
                print("\n" + "="*80 + "\n")

rag = LocalRAG(
    vector_db_config=config_helper.vector_db_config,
    embedding_config=config_helper.embedding_config, 
    chat_config=config_helper.chat_config)

with tracer.start_as_current_span("Starting demo"):
    demonstrate_local_rag(rag)